In [77]:
import pandas as pd
import numpy as np

np.random.seed(42)

In [78]:
# Standard ML Models for comparison
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

# Splitting data into training/testing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Metrics
from sklearn.metrics import mean_squared_error

In [79]:
import pymc3 as pm

In [80]:
math = pd.read_csv('data/student-mat.csv', sep=';')
por = pd.read_csv('data/student-por.csv', sep=';')
math.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [94]:
def format_data(df):
    # Targets are final grade of student
    labels = df['G3']
    # Drop the school and the grades from features
    df = df.drop(columns=['school', 'G1', 'G2', 'G3'])
    
    # One-Hot Encoding of Categorical Variables
    df = pd.get_dummies(df)
    
    # Split into training/testing sets with 25% split
    X_train, X_test, y_train, y_test = train_test_split(df, labels, 
                                                        test_size = 0.25)
    
    # Scale to between 0 and 1
    scaler = MinMaxScaler(feature_range=(0,1))
    
    # Train scaler on training data and transform all features
    # X_train = scaler.fit_transform(X_train)
    # X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test
    

In [95]:
X_train_math, X_test_math, y_train_math, y_test_math = format_data(math)

In [96]:
def evaluate(X_train, X_test, y_train, y_test):
    # Names of models
    model_name_list = ['Linear Regression', 'ElasticNet Regression',
                      'Random Forest', 'Extra Trees', 'SVM',
                       'Gradient Boosted', 'Baseline']
    
    # Instantiate the models
    model1 = LinearRegression()
    model2 = ElasticNet(alpha=1.0, l1_ratio=0.5)
    model3 = RandomForestRegressor(n_estimators=100)
    model4 = ExtraTreesRegressor(n_estimators=100)
    model5 = SVR(kernel='rbf', degree=3, C=1.0, gamma='auto')
    model6 = GradientBoostingRegressor(n_estimators=100)
    
    # Dataframe for results
    results = pd.DataFrame(columns=['rmse', 'mape'], index = model_name_list)
    
    # Train and predict with each model
    for i, model in enumerate([model1, model2, model3, model4, model5, model6]):
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        
        # Metrics
        mse = mean_squared_error(y_test, predictions)
        rmse = np.sqrt(mse)
        mape = np.median(np.divide(abs(predictions - y_test), y_test, where= y_test!=0))
        
        model_name = model_name_list[i]
        
        results.ix[model_name, :] = [rmse, mape]
    
    baseline_mse = np.mean((np.mean(y_train_math) - y_test) ** 2)
    baseline_rmse = np.sqrt(baseline_mse)
    baseline_mape = np.median(np.divide(abs(np.mean(y_train) - y_test), y_test, where= y_test!=0))
    
    results.ix['Baseline', :] = [baseline_rmse, baseline_mape]
    
    return results
        

In [97]:
math_results

,rmse,mape
Linear Regression,4.84163,0.235937
ElasticNet Regression,4.96813,0.251689
Random Forest,4.69334,0.226923
Extra Trees,4.85726,0.222857
SVM,4.87939,0.210076
Gradient Boosted,4.72427,0.224955
Baseline,4.96813,0.251689


# First Go at Linear Regression

In [98]:
X_train_math['y'] = list(y_train_math)

In [101]:
y = 'y ~'

In [117]:
formula = 'y ~ ' + ' + '.join(['%s' % variable for variable in X_train_math.columns[:-1]])

In [118]:
formula

'y ~ age + Medu + Fedu + traveltime + studytime + failures + famrel + freetime + goout + Dalc + Walc + health + absences + sex_F + sex_M + address_R + address_U + famsize_GT3 + famsize_LE3 + Pstatus_A + Pstatus_T + Mjob_at_home + Mjob_health + Mjob_other + Mjob_services + Mjob_teacher + Fjob_at_home + Fjob_health + Fjob_other + Fjob_services + Fjob_teacher + reason_course + reason_home + reason_other + reason_reputation + guardian_father + guardian_mother + guardian_other + schoolsup_no + schoolsup_yes + famsup_no + famsup_yes + paid_no + paid_yes + activities_no + activities_yes + nursery_no + nursery_yes + higher_no + higher_yes + internet_no + internet_yes + romantic_no + romantic_yes'

In [ ]:
with pm.Model() as lin_model:
    # Error term
    pm.GLM.from_formula(formula, X_train_math)
    
    lin_trace = pm.sample(draws=1000, njobs=-1)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO (theano.gof.compilelock): Refreshing lock C:\Users\Will Koehrsen\AppData\Local\Theano\compiledir_Windows-10-10.0.16299-SP0-Intel64_Family_6_Model_78_Stepping_3_GenuineIntel-3.6.4-64\lock_dir\lock
Sequential sampling (2 chains in 1 job)
NUTS: [sd_log__, romantic_yes, romantic_no, internet_yes, internet_no, higher_yes, higher_no, nursery_yes, nursery_no, activities_yes, activities_no, paid_yes, paid_no, famsup_yes, famsup_no, schoolsup_yes, schoolsup_no, guardian_other, guardian_mother, guardian_father, reason_reputation, reason_other, reason_home, reason_course, Fjob_teacher, Fjob_services, Fjob_other, Fjob_health, Fjob_at_home, Mjob_teacher, Mjob_services, Mjob_other, Mjob_health, Mjob_at_home, Pstatus_T, Pstatus_A, famsize_LE3, famsize_GT3, address_U, address_R, sex_M, sex_F, absences, health, Walc, Dalc, goout, freetime, famrel, failures, studytime, traveltime, Fedu, Medu, age, Intercept]
 96%|█████████▌